In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data = pd.read_csv("preprocessed_apartment_rentals_no_state_dummies_nocluster.csv")

# Features und Ziel
X = data.drop(columns=["price"])
y = data["price"]

# Numerische Spalten
num_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Skalieren
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
X["state"] = X["state"].astype("category")
X["cityname"] = X["cityname"].astype("category")
data.columns

Index(['bathrooms', 'bedrooms', 'has_photo', 'price', 'square_feet',
       'cityname', 'state', 'latitude', 'longitude', 'time', 'pool', 'gym',
       'parking', 'sauna', 'elevator', 'clubhouse', 'source_Andere source',
       'source_GoSection8', 'source_ListedBuy', 'source_RealRentals',
       'source_RentDigs.com', 'source_RentLingo'],
      dtype='object')

## 🧠 Hyperparameter des MLPRegressor (Neuronales Netz)

Der **MLPRegressor** implementiert ein Feedforward-Neuronales Netz und ist ein komplexes nichtlineares Modell. Die Parameter definieren das Suchgitter (`mlp_params`) für die Optimierung der Architektur und Regulierung in der Nested Cross-Validation.

| Parameter | Beschreibung | Im Code definierte Werte |
| :--- | :--- | :--- |
| **`hidden_layer_sizes`** | Definiert die **Architektur des Netzes**: Die Anzahl der versteckten Schichten (Hidden Layers) und die Anzahl der Neuronen pro Schicht. | `[(10, 10), (50, 50)]` |
| **`activation`** | Die **Aktivierungsfunktion** für die versteckten Schichten. Definiert die Nichtlinearität des Modells. | `["relu"]` |
| **`alpha`** | Der **L2-Regularisierungsterm** (äquivalent zu Ridge Regression). Ein höherer Wert reduziert die Komplexität und hilft, **Overfitting** zu verhindern. | `np.logspace(-5, -2, 4)` |
| **`max_iter`** | Die **maximale Anzahl von Iterationen** (Epochen), die der Optimierer über die Trainingsdaten ausführen soll. | `[500]` |

---

### 💡 Erläuterung der Netzarchitektur



[Image of a feedforward neural network architecture showing input, hidden, and output layers]


* **Netzarchitektur:** `(10, 10)` bedeutet zwei versteckte Schichten mit jeweils 10 Neuronen. `(50, 50)` bedeutet zwei versteckte Schichten mit jeweils 50 Neuronen.
* **`relu` (Rectified Linear Unit):** Sorgt dafür, dass das Netz **Nichtlinearitäten** lernen kann (d.h. komplexere Beziehungen als eine einfache lineare Regression).
* **`alpha` (Regularisierung):** Die Suche über $\text{np.logspace}(-5, -2, 4)$ testet logarithmisch verteilte Werte zwischen $10^{-5}$ und $10^{-2}$. Dies ist entscheidend, um die Modellkomplexität zu kontrollieren.
* **`max_iter`:** Da Neuronale Netze sequenziell lernen, ist ein ausreichender Wert notwendig, um die **Konvergenz** des Optimierers zu gewährleisten.

In [ ]:
from Nested_CV import NestedCVRegressor
from sklearn.neural_network import MLPRegressor

mlp_params = {
    "hidden_layer_sizes": [(10,10), (50, 50)],
    #"activation": ["relu"],
    #"alpha": np.logspace(-5, -2, 4),
    "max_iter": [1000], # Erhöhen der Iterationen ist oft nötig
    "batch_size": [16, 32],
    "early_stopping": [True]
}

mlp_cv = NestedCVRegressor(MLPRegressor(random_state=42), mlp_params)
print("Starte MLP Nested CV...")
mlp_cv.run(X, y, output=True)

print("\n--- MLP Ergebnisse ---")
print("MLP Mean R²:", mlp_cv.get_mean_r2())
print("MLP best params:", mlp_cv.get_best_params())
#mlp_cv.plot_scores("MLP Nested CV")

Starte MLP Nested CV...
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(10, 10), max_iter=1000; total time= 4.8min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(10, 10), max_iter=1000; total time= 2.4min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(10, 10), max_iter=1000; total time= 3.2min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(10, 10), max_iter=1000; total time= 2.8min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(10, 10), max_iter=1000; total time= 3.8min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(50, 50), max_iter=1000; total time= 3.5min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(50, 50), max_iter=1000; total time= 3.7min
[CV] END batch_size=16, early_stopping=True, hidden_layer_sizes=(50, 50), max_iter=1000; total time= 4.1min
[CV] END batch_size=16, early_stopping=True, hidden_

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Outer Fold 2/5 | Best Params: {'batch_size': 16, 'early_stopping': True, 'hidden_layer_sizes': (50, 50), 'max_iter': 1000} | Fit Time: 246.295s | Outer R²: 0.688 | MSE: 145058.973
Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [ ]:
print(mlp_cv.get_best_params())

In [ ]:
import pandas as pd
number_outer_cv_splits = 5
model_evaluation = pd.DataFrame({"Fold": range(1, number_outer_cv_splits+1)})
model_evaluation['Modell'] = ['MLP_NN'] * number_outer_cv_splits
model_evaluation['R_2'] = mlp_cv.get_r2_scores()
model_evaluation['MSE'] = mlp_cv.get_mse_scores()
model_evaluation["runtime"] = mlp_cv.get_fit_times()
model_evaluation

# TORCH NN

In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin

class TorchRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, hidden_sizes=(64, 64), lr=1e-3, epochs=50, batch_size=64, device=None):
        self.hidden_sizes = hidden_sizes
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

    def _build_model(self, input_dim):
        layers = []
        last = input_dim
        for h in self.hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, 1))
        return nn.Sequential(*layers)

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(self.device)

        self.model = self._build_model(X.shape[1]).to(self.device)
        opt = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        loss_fn = nn.MSELoss()

        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        self.model.train()
        for _ in range(self.epochs):
            for xb, yb in loader:
                opt.zero_grad()
                pred = self.model(xb)
                loss = loss_fn(pred, yb)
                loss.backward()
                opt.step()

        return self

    def predict(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values
        if isinstance(y, pd.Series):
            y = y.values

        self.model.eval()
        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            return self.model(X).cpu().numpy().ravel()


In [8]:
from Nested_CV_targetEncoding import NestedCVRegressorWithTargetEncoding

torch_params = {
    "hidden_sizes": [(32,32), (64,64)],
    "epochs": [500],
    "batch_size": [32],
    #"lr": [1e-3],
}

torch_cv = NestedCVRegressorWithTargetEncoding(TorchRegressor(), param_grid=torch_params, encode_cols=["state", "cityname"], outer_splits=2, inner_splits=2)

print("Starte Torch Nested CV...")
torch_cv.run(X, y, output=True)

print("\n--- Torch NN Ergebnisse ---")
print("Mean R²:", torch_cv.get_mean_r2())
print("Best params:", torch_cv.get_best_params())


Starte Torch Nested CV...
Fitting 2 folds for each of 2 candidates, totalling 4 fits


KeyboardInterrupt: 

In [ ]:
import pandas as pd
number_outer_cv_splits = 5
model_evaluation = pd.DataFrame({"Fold": range(1, number_outer_cv_splits+1)})
model_evaluation['Modell'] = ['Torch_NN'] * number_outer_cv_splits
model_evaluation['R_2'] = torch_cv.get_r2_scores()
model_evaluation['MSE'] = torch_cv.get_mse_scores()
model_evaluation["runtime"] = torch_cv.get_fit_times()
model_evaluation